In [17]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from statsmodels.multivariate.manova import MANOVA

In [16]:
df = pd.read_csv("Removed_outliers_byGroup_data.csv")
df['group'] = df.apply(lambda row: f"{int(row['HINPOV'])},{int(row['PENINC'])},{int(row['HIGOV'])},{int(row['RETMON'])},{int(row['SLFEMP'])}", axis=1)
groups = df['group'].unique()
print(groups)
continuous_features = ['BMI', 'INHPFN',  'HHHRES', 'HCHILD','LIVSIB',  'HAIRA', 'HATOTB', 'IEARN','HITOT', 'PRPCNT']
cate_features = ['HINPOV','PENINC', 'HIGOV','RETMON', 'SLFEMP']
target = ["SHLT","MSTOT","COGTOT"]
df = df.drop(cate_features, axis = 1)
df

['0,0,0,0,0' '0,0,0,0,1' '0,0,1,0,0' '0,0,1,0,1' '0,0,1,1,0' '0,0,1,1,1'
 '0,1,1,0,0' '0,1,1,1,0' '0,1,1,1,1' '1,0,0,0,0']


,SHLT,BMI,MSTOT,COGTOT,INHPFN,HHHRES,HCHILD,LIVSIB,HAIRA,HATOTB,IEARN,HITOT,PRPCNT,group
0,5.0,33.0,14.0,17.0,0.0,2.0,4.0,0.0,0.0,0.0,20000.0,22400.0,0.0,"0,0,0,0,0"
1,4.0,23.8,8.0,14.0,0.0,2.0,6.0,2.0,0.0,15000.0,25000.0,107000.0,1.0,"0,0,0,0,0"
2,3.0,26.0,15.0,27.0,0.0,2.0,2.0,1.0,40000.0,290000.0,103000.0,134384.0,1.0,"0,0,0,0,0"
3,4.0,40.7,11.0,16.0,0.0,3.0,4.0,7.0,0.0,16477.0,62000.0,72157.0,0.0,"0,0,0,0,0"
4,3.0,22.8,15.0,31.0,0.0,4.0,4.0,4.0,4000.0,138300.0,15000.0,95660.0,1.0,"0,0,0,0,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,4.0,39.6,14.0,23.0,0.0,2.0,5.0,6.0,0.0,27500.0,27000.0,29112.0,0.0,"1,0,0,0,0"
33534,1.0,18.4,14.0,27.0,0.0,5.0,6.0,5.0,0.0,90000.0,0.0,0.0,1.0,"1,0,0,0,0"
33535,1.0,26.9,9.0,25.0,0.0,2.0,7.0,7.0,0.0,2341.0,3295.0,3295.0,1.0,"1,0,0,0,0"
33536,4.0,29.3,13.0,23.0,0.0,3.0,3.0,7.0,0.0,0.0,18000.0,18000.0,0.0,"1,0,0,0,0"


In [13]:
for group in groups:
    print(df[df["group"]==group][target].corr())

            SHLT     MSTOT    COGTOT
SHLT    1.000000 -0.167408 -0.189673
MSTOT  -0.167408  1.000000  0.668407
COGTOT -0.189673  0.668407  1.000000
            SHLT     MSTOT    COGTOT
SHLT    1.000000 -0.168911 -0.189017
MSTOT  -0.168911  1.000000  0.685551
COGTOT -0.189017  0.685551  1.000000
            SHLT     MSTOT    COGTOT
SHLT    1.000000 -0.132131 -0.206216
MSTOT  -0.132131  1.000000  0.666644
COGTOT -0.206216  0.666644  1.000000
            SHLT     MSTOT    COGTOT
SHLT    1.000000 -0.098731 -0.188849
MSTOT  -0.098731  1.000000  0.586799
COGTOT -0.188849  0.586799  1.000000
            SHLT     MSTOT    COGTOT
SHLT    1.000000 -0.114583 -0.178451
MSTOT  -0.114583  1.000000  0.620651
COGTOT -0.178451  0.620651  1.000000
            SHLT     MSTOT    COGTOT
SHLT    1.000000 -0.141674 -0.154367
MSTOT  -0.141674  1.000000  0.585759
COGTOT -0.154367  0.585759  1.000000
            SHLT     MSTOT    COGTOT
SHLT    1.000000 -0.107435 -0.150123
MSTOT  -0.107435  1.000000  0.608581
C

In [21]:
maov = MANOVA.from_formula('SHLT+MSTOT+COGTOT ~ group', data=df)

maov.mv_test().summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                     Multivariate linear model
====================================================================
                                                                    
--------------------------------------------------------------------
       Intercept         Value  Num DF   Den DF     F Value   Pr > F
--------------------------------------------------------------------
          Wilks' lambda  0.0334 3.0000 33526.0000 323241.5596 0.0000
         Pillai's trace  0.9666 3.0000 33526.0000 323241.5596 0.0000
 Hotelling-Lawley trace 28.9246 3.0000 33526.0000 323241.5596 0.0000
    Roy's greatest root 28.9246 3.0000 33526.0000 323241.5596 0.0000
--------------------------------------------------------------------
                                                                    
--------------------------------------------------------------------
           group          Value   Num DF    Den DF   F Value  Pr > F
--------------------------------------------------------------------
            Wilks' lambda 0.9473 27.0000  97913.8506  67.9149 0.0000
           Pillai's trace 0.0535 27.0000 100584.0000  67.6003 0.0000
   Hotelling-Lawley trace 0.0549 27.0000  74781.1009  68.1960 0.0000
      Roy's greatest root 0.0320  9.0000  33528.0000 119.2822 0.0000
====================================================================

"""

In [32]:
pairwise_combinations = [(groups[i], groups[j]) for i in range(len(groups)) for j in range(i+1, len(groups))]
significant_pairs = []

for group1, group2 in pairwise_combinations:
    temp_df = df[df['group'].isin([group1, group2])]
    
    maov = MANOVA.from_formula('SHLT+MSTOT+COGTOT ~ group', data=temp_df)

    result = maov.mv_test()
    print(f"MANOVA between {group1} and {group2}:")
    
    group_p_value = result.results['group']['stat']['Pr > F'][0]
    print(f"P-value: {group_p_value}")
    
    if group_p_value > 0.1:
        significant_pairs.append((group1, group2))

print("Significant group pairs:", significant_pairs)


MANOVA between 0,0,0,0,0 and 0,0,0,0,1:
P-value: 1.0128320048164901e-12
MANOVA between 0,0,0,0,0 and 0,0,1,0,0:
P-value: 1.1509202714085733e-36
MANOVA between 0,0,0,0,0 and 0,0,1,0,1:
P-value: 7.519252017116213e-73
MANOVA between 0,0,0,0,0 and 0,0,1,1,0:
P-value: 2.0808869823462375e-81
MANOVA between 0,0,0,0,0 and 0,0,1,1,1:
P-value: 7.991715566316587e-71
MANOVA between 0,0,0,0,0 and 0,1,1,0,0:
P-value: 5.512406700693228e-20
MANOVA between 0,0,0,0,0 and 0,1,1,1,0:
P-value: 6.441421145250314e-61
MANOVA between 0,0,0,0,0 and 0,1,1,1,1:
P-value: 4.416899659084358e-52
MANOVA between 0,0,0,0,0 and 1,0,0,0,0:
P-value: 1.498656725832206e-109
MANOVA between 0,0,0,0,1 and 0,0,1,0,0:
P-value: 7.151692526131521e-39
MANOVA between 0,0,0,0,1 and 0,0,1,0,1:
P-value: 4.3698417959793267e-44
MANOVA between 0,0,0,0,1 and 0,0,1,1,0:
P-value: 2.0110937805851863e-57
MANOVA between 0,0,0,0,1 and 0,0,1,1,1:
P-value: 2.9064296902981532e-53
MANOVA between 0,0,0,0,1 and 0,1,1,0,0:
P-value: 1.7125030612740796e-1